In [1]:
import pymongo
import bson
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [4]:
def reserved(bson_id):
#     print("BEFORE:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["status"])
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                        {"$set":{"status":"reserved",
                                                "reason:":"experiment_with_industries"}})
#     print("AFTER:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["status"])

In [6]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"status":"pipelined"}):
    if "industries_from_bert" in i and i["industries_from_bert"]:
        pass
        counter += 1
    else:
        reserved(i["_id"])
#         print(i)
counter

822

In [8]:
db_client["tweets_pipeline_v2"].find({"status":"pipelined"}).count()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


822

In [11]:
tags_stats = []
for i in db_client["tweets_pipeline_v2"].find({"status":"pipelined"}):
    tags_stats += i["bert_tags"]+i["entity_prepared"]

In [12]:
from collections import Counter

In [14]:
Counter(tags_stats).most_common()

[('commodities', 400),
 ('us', 267),
 ('health', 167),
 ('retail', 160),
 ('energy', 141),
 ('global', 140),
 ('pandemic', 140),
 ('covid-19', 138),
 ('coronavirus', 137),
 ('index', 128),
 ('banking', 121),
 ('oil', 116),
 ('real estate', 102),
 ('mining', 102),
 ('metals', 96),
 ('bonds', 89),
 ('financials', 80),
 ('loan', 76),
 ('gdp', 76),
 ('diseases', 75),
 ('china', 75),
 ('trade', 75),
 ('death', 73),
 ('uk', 71),
 ('technology', 68),
 ('labor', 68),
 ('manufacturing', 66),
 ('consumer', 65),
 ('debt', 60),
 ('transportation', 56),
 ('medicine', 56),
 ('automotive', 55),
 ('government bonds', 52),
 ('s&p', 47),
 ('germany', 46),
 ('credit', 46),
 ('employment', 46),
 ('services', 45),
 ('social', 43),
 ('vehicles', 42),
 ('crisis', 42),
 ('italy', 41),
 ('macro', 40),
 ('japan', 40),
 ('france', 39),
 ('spain', 39),
 ('fed', 38),
 ('household', 33),
 ('government', 33),
 ('funds', 32),
 ('priceline', 31),
 ('mortgage', 30),
 ('gold', 30),
 ('canada', 30),
 ('education', 30),
 

In [34]:
from datetime import datetime

In [35]:
def updateAPP(bson_id, new_list):
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
#     print(bson_id)
    print("BEFORE:", before["status"])
    date = datetime.now()
    ########################## CHANGES ######################
    status = "graphicone_search"
#     print()
#     print("status", status)
#     print("approved_timestamp", [date])
    print("grafeed_confirmed", new_list)
#     print("additional_info", "moderator passed")
#     print("grafeed_approver", "VasiliiUshakov")
#     print()
    
    
    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                    {"$set":{"status": status,
                                             "approved_timestamp": date,
                                             "grafeed_confirmed": new_list,
                                             "status_before": before["status"],
                                             "additional_info": "moderator tool passed",
                                             "grafeed_approver": "VasiliiUshakov",
                                            }})
    
    

    after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
    print("AFTER:", after["status"], after["approved_timestamp"], after["grafeed_confirmed"], after["additional_info"], after["grafeed_approver"])
    print("AFTER:", new_list)
    print()     

In [38]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"validated"},
                                                      ],
                                              "industries_from_bert": {"$exists": True}}):
    if not i["industries_from_bert"] and i["validator_username"] in ["tagger48", "alyona"]:
#         print(i["_id"], i["confirmed_after_validate"])
        tags = [row.lower() for row in i["confirmed_after_validate"]]
        updateAPP(i["_id"], tags)
        counter += 1
counter

0

In [48]:
counter = 0
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},
                                                      ],
                                               "additional_info": {"$exists": True},
                                              "industries_from_bert": {"$exists": True}}):
    if not i["industries_from_bert"] and i["validator_username"] in ["tagger48", "alyona"]:
        tags = [row.lower() for row in i["grafeed_confirmed"]]
        updateAPP(i["_id"], tags)
        counter += 1
counter

BEFORE: graphicone_search
grafeed_confirmed ['us', 'foreign investment']
AFTER: graphicone_search 2020-05-06 12:21:26.769000 ['us', 'foreign investment'] moderator tool passed VasiliiUshakov
AFTER: ['us', 'foreign investment']

BEFORE: graphicone_search
grafeed_confirmed ['travel', 'new south wales', 'abu dhabi', 'tasmania', 'cities', 'wuhan', 'florida', 'adelaide', 'us', 'australia', 'china', 'uae']
AFTER: graphicone_search 2020-05-06 12:21:26.853000 ['travel', 'new south wales', 'abu dhabi', 'tasmania', 'cities', 'wuhan', 'florida', 'adelaide', 'us', 'australia', 'china', 'uae'] moderator tool passed VasiliiUshakov
AFTER: ['travel', 'new south wales', 'abu dhabi', 'tasmania', 'cities', 'wuhan', 'florida', 'adelaide', 'us', 'australia', 'china', 'uae']

BEFORE: graphicone_search
grafeed_confirmed ['gdp', 'macro', 'eu', 'pmi', 'index']
AFTER: graphicone_search 2020-05-06 12:21:26.937000 ['gdp', 'macro', 'eu', 'pmi', 'index'] moderator tool passed VasiliiUshakov
AFTER: ['gdp', 'macro', 

AFTER: graphicone_search 2020-05-06 12:21:28.957000 ['pandemic', 'coronavirus', 'covid-19', 'disease', 'ecuador', 'jpm', 'argentina'] moderator tool passed VasiliiUshakov
AFTER: ['pandemic', 'coronavirus', 'covid-19', 'disease', 'ecuador', 'jpm', 'argentina']

BEFORE: graphicone_search
grafeed_confirmed ['buyback', 'gdp', 'government', 'legal']
AFTER: graphicone_search 2020-05-06 12:21:29.041000 ['buyback', 'gdp', 'government', 'legal'] moderator tool passed VasiliiUshakov
AFTER: ['buyback', 'gdp', 'government', 'legal']

BEFORE: graphicone_search
grafeed_confirmed ['uk']
AFTER: graphicone_search 2020-05-06 12:21:29.125000 ['uk'] moderator tool passed VasiliiUshakov
AFTER: ['uk']

BEFORE: graphicone_search
grafeed_confirmed ['s&p', 'index']
AFTER: graphicone_search 2020-05-06 12:21:29.209000 ['s&p', 'index'] moderator tool passed VasiliiUshakov
AFTER: ['s&p', 'index']

BEFORE: graphicone_search
grafeed_confirmed ['employment', 'labor', 'china']
AFTER: graphicone_search 2020-05-06 12:21

BEFORE: graphicone_search
grafeed_confirmed ['employment', 'labor', 'macro', 'recession', 'us']
AFTER: graphicone_search 2020-05-06 12:21:31.484000 ['employment', 'labor', 'macro', 'recession', 'us'] moderator tool passed VasiliiUshakov
AFTER: ['employment', 'labor', 'macro', 'recession', 'us']

BEFORE: graphicone_search
grafeed_confirmed ['employment', 'labor', 'macro', 'recession', 'us']
AFTER: graphicone_search 2020-05-06 12:21:31.568000 ['employment', 'labor', 'macro', 'recession', 'us'] moderator tool passed VasiliiUshakov
AFTER: ['employment', 'labor', 'macro', 'recession', 'us']

BEFORE: graphicone_search
grafeed_confirmed ['employment', 'labor', 'population', 'norway', 'sweden']
AFTER: graphicone_search 2020-05-06 12:21:31.652000 ['employment', 'labor', 'population', 'norway', 'sweden'] moderator tool passed VasiliiUshakov
AFTER: ['employment', 'labor', 'population', 'norway', 'sweden']

BEFORE: graphicone_search
grafeed_confirmed ['large caps', 's&p', 'index']
AFTER: graphicon

AFTER: graphicone_search 2020-05-06 12:21:33.671000 ['cash', 'us', 'government'] moderator tool passed VasiliiUshakov
AFTER: ['cash', 'us', 'government']

BEFORE: graphicone_search
grafeed_confirmed ['inflation', 'cpi', 'index', 'us', 'cities']
AFTER: graphicone_search 2020-05-06 12:21:34.594000 ['inflation', 'cpi', 'index', 'us', 'cities'] moderator tool passed VasiliiUshakov
AFTER: ['inflation', 'cpi', 'index', 'us', 'cities']

BEFORE: graphicone_search
grafeed_confirmed ['debt', 'global', 'financials', 'government', 'government debt']
AFTER: graphicone_search 2020-05-06 12:21:34.678000 ['debt', 'global', 'financials', 'government', 'government debt'] moderator tool passed VasiliiUshakov
AFTER: ['debt', 'global', 'financials', 'government', 'government debt']

BEFORE: graphicone_search
grafeed_confirmed ['death', 'us', 'new york', 'terrorism', 'covid-19', 'pandemic', 'coronavirus']
AFTER: graphicone_search 2020-05-06 12:21:34.762000 ['death', 'us', 'new york', 'terrorism', 'covid-19'

92

In [28]:
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},
                                                       {"status":"graphicone_feed"}
                                                      ]}):
    print(i)

{'_id': ObjectId('5dc4c2565dcf6274b6a44809'), 'tweet_text': "The World's 20 Most Profitable Companies 💵\nhttps://t.co/fY7YXoAixd via @VisualCap #money #business #economy #wealth https://t.co/GPqtI7LVLY", 'hashtags': ['money', 'business', 'economy', 'wealth'], 'tweet_timestamp': datetime.datetime(2019, 10, 27, 16, 2), 'username': 'tamaramccleary', 'user_id': 2307675307, 'user_counters': {'followers_count': 306374, 'friends_count': 215979, 'listed_count': 8590}, 'tweet_counters': {'retweet_count': 11, 'favorite_count': 11}, 'img_urls': 'http://pbs.twimg.com/media/EHz92SGVUAAXDDB.jpg', 'tweet_id': 1188486009236836358, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'graph_type': 'other-chart', 'confirmed_after_validate': ['saudi aramco', 'apple', 'icbc', 'samsung', 'china construction bank', 'jpmorgan', 'alphabet', 'bank of america', 'shelll', 'shell', 'wells fargo', 'gazprom', 'facebook', 'intel', 'exxonmobil', 'at&t', 'citi', 'toyota'], 'status': 'graphicone_feed', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('5e90b005eb01568cd3529325'), 'tweet_text': 'RT @wef: Will the coronavirus break the internet? https://t.co/vLTSkLGeQb #coronavirus #COVID19 https://t.co/2CDz6KBZEY', 'hashtags': ['coronavirus', 'covid19'], 'username': 'yglvoices', 'tweet_timestamp': datetime.datetime(2020, 3, 24, 8, 53, 3), 'tweet_id': 1242373852371603457, 'img_urls': 'https://pbs.twimg.com/media/ET1t9EtWoAAtkV4.png', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 3, 24, 8, 53, 3), 'source': 'twitter', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'graphicone_search', 'extracted_text': 'Mobile Broadband Performance in Select Asian Countries 100 Speedtest® Data 80 China 60 Hubei, China India Japan Malaysia 40 20 Dec 16, 2019 Dec 30 Jan 6, 2020 Jan 13 Jan 20 Jan 27 Feb 3 Feb 10 Feb 17 Mar 2 Mar 9 Mean Download Speed (Mbps) Dec 23 Feb 24', 'bert_tags': ['mobile', 'technology', 'telecom'], 'industries_from_bert': ['Networks and communications', 'Technology as you know it'], 'shifts_from_bert': [